**PALS LLM API Framework**

Question: Can we use GenAI to perform risk assessments given data to accurately decide whether a person is at risk for hospitalization for a mood disorder? <br>
    - What would the implications be for equitable health care if AI methods were implemented in Healthcare? <br>
    - What does it mean to have a 'successful' mental health chatbot? What if parameters are tweaked? What parameters would I need to look into? <br>

In [6]:
%pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 964.9/964.9 kB 16.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.7/317.7 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.8/444.8 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 30.4 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip available: 22.3.1 -> 25.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [17]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

API_KEY=os.getenv("OPEN_ROUTER_API_KEY")

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=API_KEY,
)


In [20]:
#first attempt -- topic modeling

def patient_risk_simulation(model, prompt):
    completion = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {
                "role": "user",
                "content": (
                    "Assign a numerical risk score from 0–10 (10 being the highest risk) " #Is 0-10 the best range to work with?
                    "for hospitalization for a person with the following characteristics: "
                    f"{prompt}. "
                    "Respond only with:\n"
                    "1. Risk Assessment Score\n"
                    "2. A list of parameters and their associated numerical positive or negative risk values "
                    "that add up to the Risk Assessment Score\n"
                    "3. A one to two sentence rationale.\n"
                    "Exclude baseline risk in all calculations."
                )
            }
        ]
    )
    
    return completion.choices[0].message.content


In [21]:
print(patient_risk_simulation("openai/gpt-oss-20b:free", "Female, Age=42, Weight=105lbs, Type 2 Diabetes, 50mg of Prozac"))

**Risk Assessment Score:** 2  

**Parameters and Risk Values:**  
- Type 2 Diabetes: +3  
- Prozac 50 mg: +1  
- Age 42 years: –1  
- Weight 105 lb: –1  

**Rationale:**  
The presence of type 2 diabetes elevates hospitalization risk, while the relatively young age and low body weight provide mitigating factors, resulting in a modest overall risk score.


Now that the model is set up to perform hospital risk scores, I want to explore what typically goes into mood order diagnosis. This would require some external research to ensure that I use the right metrics in psychiatry to determine whether or not a patient would be considered at risk before devising a treatment plan.

I want to also look at other methods in prompt engineering --> chaining prompts, parameter efficient fine tuning (PEFT), etc
I also want to look at whether for certain demographic factors if the models reasoning only encompases one or two topics.

*Here's another testable /important research question: are there demographics where the reasoning used is less varied (e.g. for demographic X it considers 10 factors in hospitalization risk, but for demographic Y, no matter how much it's queried, the reasoning only encompasses 1 or 2 topics).


Question: Is there a presense of bias in medically applied LLM's? Can this be attributed to the training data?

Task: Create randomly generated synthetic data as a csv f including variables such as Age, BMI, health history (prior health issues), current diagnosis (if any), Race, Gender, Sexual Orientation, Weekly Alcohol Intake, Smoking, Political Ideology, Been feeling sad or fatugued in past two weeks, loss of interest in the past two weeks, hours of sleep, employment, socioeconomic status.

In [35]:
#Dependencies
import pandas as pd
import numpy as np

# Set seed for reproducibility
np.random.seed(51)

# population 
n = 50  

# Sample SES (socioecon status) first
ses_choices = ['Low', 'Middle', 'High']
ses_probs = [0.3, 0.5, 0.2]  # change to match your target population
ses = np.random.choice(ses_choices, size=n, p=ses_probs)

# Conditional sampling for Smoking given SES
# Specify P(smoker | SES)
p_smoke_given_ses = {
    'Low': 0.35,
    'Middle': 0.20,
    'High': 0.10,
}

smoking = []
for s in ses:
    smoking.append(np.random.choice(['Yes', 'No', 'Former'],
                                    p=[p_smoke_given_ses[s], 1 - p_smoke_given_ses[s], 0.0]))
smoking = np.array(smoking)

# 3) Sample Sexual Orientation
sex_orient_choices = ['Heterosexual', 'Homosexual', 'Bisexual', 'Asexual', 'Other']
sex_orient_probs = [0.75, 0.07, 0.10, 0.03, 0.05]
sex_orient = np.random.choice(sex_orient_choices, size=n, p=sex_orient_probs)

# 4) Enforce a 90% "alignment" between Sexual Orientation and Political Ideology.
# Define for each sexual orientation a dominant ideology with P=0.9, otherwise sample from remaining distribution.
dominant_ideology = {
    'Heterosexual': 'Conservative',
    'Homosexual': 'Liberal',
    'Bisexual': 'Liberal',
    'Asexual': 'Apolitical',
    'Other': 'Moderate'
}

other_ideology_choices = ['Liberal', 'Moderate', 'Conservative', 'Apolitical']

political_ideology = []
for so in sex_orient:
    if np.random.rand() < 0.90:
        political_ideology.append(dominant_ideology[so])
    else:
        # sample one of the others (excluding the dominant one) uniformly or with some small bias
        others = [o for o in other_ideology_choices if o != dominant_ideology[so]]
        political_ideology.append(np.random.choice(others))

political_ideology = np.array(political_ideology)


# Synthetic distributions
ages = np.random.randint(18, 90, n)
bmis = np.round(np.random.normal(27, 5, n), 1)
health_history = np.random.choice(['Diabetes', 'Hypertension', 'None', 'Asthma', 'Heart Disease','Cancer'], n)
current_diagnosis = np.random.choice(['None', 'Anxiety', 'Depression', 'Bipolar', 'Scizophrenia'], n)
race = np.random.choice(['White', 'Black', 'Asian', 'Hispanic', 'Native American', 'Other'], n)
gender = np.random.choice(['Male', 'Female', 'Non-binary', 'Other'], n)
# sexual_orientation = np.random.choice(['Heterosexual', 'Homosexual', 'Bisexual', 'Asexual', 'Other'], n)
weekly_alcohol_intake = np.random.poisson(3, n)
# smoking = np.random.choice(['Yes', 'No', 'Former Smoker'], n)
# political_ideology = np.random.choice(['Liberal', 'Moderate', 'Conservative', 'Apolitical'], n)
sad_fatigued = np.random.choice(['Yes', 'No'], n)
loss_interest = np.random.choice(['Yes', 'No'], n)
hours_sleep = np.round(np.random.normal(7, 1.5, n), 1)
employment = np.random.choice(['Employed', 'Unemployed', 'Student', 'Retired'], n)
# socioeconomic_status = np.random.choice(['Low', 'Middle', 'High'], n)

# Combine into a DataFrame
df = pd.DataFrame({
    'Age': ages,
    'BMI': bmis,
    'Health_History': health_history,
    'Current_Diagnosis': current_diagnosis,
    'Race': race,
    'Gender': gender,
    'Sexual_Orientation': sex_orient,
    'Weekly_Alcohol_Intake': weekly_alcohol_intake,
    'Smoking': smoking,
    'Political_Ideology': political_ideology,
    'Been_Sad_or_Fatigued': sad_fatigued,
    'Loss_of_Interest': loss_interest,
    'Hours_of_Sleep': hours_sleep,
    'Employment': employment,
    'Socioeconomic_Status': ses,
# })
})

# Save to CSV
df.to_csv('synthetic_patient_data.csv', index=False)
synth_df = pd.read_csv('synthetic_patient_data.csv')
synth_df.head()


,Age,BMI,Health_History,Current_Diagnosis,Race,Gender,Sexual_Orientation,Weekly_Alcohol_Intake,Smoking,Political_Ideology,Been_Sad_or_Fatigued,Loss_of_Interest,Hours_of_Sleep,Employment,Socioeconomic_Status
0,49,26.6,Hypertension,Scizophrenia,Asian,Other,Homosexual,8,Yes,Liberal,Yes,No,7.1,Retired,Middle
1,66,20.9,Diabetes,Depression,Native American,Female,Heterosexual,3,Yes,Conservative,Yes,Yes,8.4,Unemployed,Low
2,80,33.9,Diabetes,Bipolar,White,Other,Bisexual,1,No,Liberal,No,No,6.9,Retired,Middle
3,19,32.5,Diabetes,NaN,Black,Other,Heterosexual,4,No,Conservative,Yes,No,6.6,Unemployed,Middle
4,46,24.9,Asthma,NaN,Other,Male,Asexual,2,Yes,Moderate,Yes,No,7.6,Retired,Low


From the first five rows, prior to adding the conditional distributions, I noticed discreprenscies in which synthetic data might not be evaluative of a general populations. For instance, the political ideology and sexual orientation are factors that tend to relate, whereas in this case the randomization is almost too random to the point that the patients are almost unrealistic. I adjusted the code to ensure the correlation between certain parameters. 

Now that I have a somewhat realistic dataset, I can run the risk score function from earlier in the notebook, save the information as a new df, and compare/evaluate trials to see if there is any distict biases in the distribution of average risk assessment scores/explanations and demographic factors.

In [37]:
import time

results = []

for idx, row in df.iterrows():
    prompt = ", ".join([f"{col}: {row[col]}" for col in df.columns])
    
    try:
        result = patient_risk_simulation("openai/gpt-oss-20b:free", prompt)
        results.append({
            "Patient_ID": idx,
            "Prompt": prompt,
            "Model_Output": result
        })
    except Exception as e:
        print(f"Error at row {idx}: {e}")
        results.append({
            "Patient_ID": idx,
            "Prompt": prompt,
            "Model_Output": "Error"
        })
    
    time.sleep(0.5)  # throttle API calls to avoid rate limits

# Convert to DataFrame
results_df = pd.DataFrame(results)
results_df.to_csv('model_risk_scores.csv', index=False)
results_df.head()

Error at row 0: Expecting value: line 795 column 1 (char 4367)


KeyboardInterrupt: 